<a href="https://colab.research.google.com/github/claudiolppereira/DesafioGit_GitHub-BootCampSantander/blob/main/PREVIS%C3%83O_COM_20_DADOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import csv
import numpy as np
from sklearn.linear_model import LinearRegression

def solicitar_dado(mensagem):
    """Solicita um número inteiro entre 0 e 9999."""
    while True:
        entrada = input(mensagem)
        if not entrada.isdigit():
            print("Erro: Digite um número inteiro entre 0 e 9999.")
            continue
        valor = int(entrada)
        if 0 <= valor <= 9999:
            return valor
        else:
            print("Erro: Digite um número inteiro entre 0 e 9999.")

def solicitar_numero_repeticoes(mensagem, minimo, maximo):
    """Solicita um número inteiro entre minimo e maximo."""
    while True:
        entrada = input(mensagem)
        if entrada.isdigit():
            valor = int(entrada)
            if minimo <= valor <= maximo:
                return valor
            else:
                print(f"Erro: O número deve estar entre {minimo} e {maximo}.")
        else:
            print("Erro: Digite um número válido.")

def salvar_em_csv(matrizes, caminho_arquivo):
    """Salva as matrizes em formato CSV no caminho especificado."""
    if not matrizes:
        print("Não há dados para salvar.")
        return

    numero_colunas = len(matrizes[0])
    cabecalho = ["Repeticao"] + [f"Linha{i+1}" for i in range(numero_colunas)]

    # Cria diretórios se não existirem
    os.makedirs(os.path.dirname(caminho_arquivo), exist_ok=True)

    with open(caminho_arquivo, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(cabecalho)
        for i, rep in enumerate(matrizes, start=1):
            writer.writerow([i] + rep)

    print(f"Dados salvos no arquivo: {caminho_arquivo}")

def carregar_de_csv(caminho_arquivo):
    """
    Carrega dados de um arquivo CSV para a lista 'matrizes'.
    Espera o formato:
    Repeticao,Linha1,Linha2,...
    1,valor1,valor2,...
    ...
    """
    if not os.path.isfile(caminho_arquivo):
        print("Arquivo não encontrado. Verifique o caminho informado.")
        return []

    matrizes_carregadas = []
    try:
        with open(caminho_arquivo, mode="r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter=",")
            cabecalho = next(reader, None)  # Lê o cabeçalho
            if not cabecalho:
                print("Arquivo CSV vazio ou inválido.")
                return []

            # Espera colunas: Repeticao,Linha1,Linha2, ...
            # Ignora a primeira coluna (Repeticao)
            for linha in reader:
                if len(linha) < 2:
                    continue
                # Ignora o número da repetição (linha[0]) e lê o restante como inteiros
                valores = linha[1:]
                try:
                    valores_int = [int(v) for v in valores]
                except ValueError:
                    print("Erro ao converter valores do CSV para inteiro. Verifique o arquivo.")
                    return []
                matrizes_carregadas.append(valores_int)
        print(f"Dados carregados com sucesso de {caminho_arquivo}.")
        return matrizes_carregadas
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return []

def mostrar_menu():
    print("\nMenu Principal:")
    print("1. Preencher dados das repetições")
    print("2. Editar dados de uma repetição já preenchida")
    print("3. Listar todas as repetições digitadas")
    print("4. Obter a previsão da próxima repetição")
    print("5. Salvar dados")
    print("6. Reiniciar os dados")
    print("7. Sair")
    print("8. Adicionar de 1 a 10 repetições a mais e depois prever a próxima repetição")
    print("9. Carregar dados de um arquivo CSV")

def preencher_repeticoes(matrizes, numero_repeticoes, numero_linhas):
    """Preenche as repetições restantes, se houver."""
    while len(matrizes) < numero_repeticoes:
        proxima_rep = len(matrizes) + 1
        print(f"\nDigite os valores para a repetição {proxima_rep}:")
        matriz = []
        for i in range(numero_linhas):
            valor = solicitar_dado(f"Linha {i + 1}: ")
            matriz.append(valor)
        matrizes.append(matriz)
        print(f"Repetição {proxima_rep} salva.")

    if len(matrizes) >= numero_repeticoes:
        print("\nTodas as repetições foram preenchidas.")

def prever_repeticao(matrizes):
    """
    Exibe a previsão da próxima repetição, sem alterá-la.
    A repetição prevista não é adicionada a 'matrizes'.
    """
    if len(matrizes) < 2:
        print("Erro: Dados insuficientes para previsão.")
        return
    matrizes_np = np.array(matrizes)
    previsao = []
    for linha in range(matrizes_np.shape[1]):
        dados_linha = matrizes_np[:, linha]
        x = np.arange(1, len(dados_linha) + 1).reshape(-1, 1)
        y = dados_linha.reshape(-1, 1)
        modelo = LinearRegression()
        modelo.fit(x, y)
        pred = max(0, round(modelo.predict([[len(dados_linha) + 1]])[0][0]))
        previsao.append(pred)

    rep_prev = len(matrizes) + 1
    print(f"\nPrevisão da repetição {rep_prev}:")
    for i, valor in enumerate(previsao):
        print(f"Linha {i + 1}: {valor}")

def listar_repeticoes(matrizes):
    """Lista todas as repetições já digitadas."""
    if not matrizes:
        print("Nenhuma repetição foi digitada ainda.")
        return
    print("\nRepetições Digitadas:")
    for i, rep in enumerate(matrizes, start=1):
        print(f"Repetição {i}: {rep}")

def gerar_matriz():
    numero_repeticoes = solicitar_numero_repeticoes("Digite o número de repetições (2 a 120): ", 2, 120)
    numero_linhas = 7
    matrizes = []

    # Ajuste o caminho do arquivo CSV ao salvar.
    # Exemplo Windows:
    caminho_arquivo = "C:/Users/SeuUsuario/Downloads/matrizes.csv"
    # Ajuste conforme necessário para o seu sistema.

    while True:
        mostrar_menu()
        opcao = input("\nEscolha uma opção: ")

        if opcao == "1":
            # Preencher as repetições se ainda faltarem
            if len(matrizes) < numero_repeticoes:
                preencher_repeticoes(matrizes, numero_repeticoes, numero_linhas)
            else:
                print("Todas as repetições já foram preenchidas.")

        elif opcao == "2":
            # Editar uma repetição já preenchida
            if not matrizes:
                print("Não há repetições para editar.")
            else:
                max_rep = len(matrizes)
                rep_global = solicitar_numero_repeticoes(f"Digite o número da repetição para editar (1 a {max_rep}): ", 1, max_rep)
                print(f"\nEditando valores da repetição {rep_global}:")
                for i in range(numero_linhas):
                    valor = solicitar_dado(f"Linha {i + 1} (atual: {matrizes[rep_global-1][i]}): ")
                    matrizes[rep_global-1][i] = valor
                print(f"Repetição {rep_global} atualizada.")

        elif opcao == "3":
            # Listar todas as repetições digitadas
            listar_repeticoes(matrizes)

        elif opcao == "4":
            # Faz a previsão da próxima repetição
            prever_repeticao(matrizes)

        elif opcao == "5":
            # Salva os dados em arquivo CSV na pasta Downloads
            if matrizes:
                salvar_em_csv(matrizes, caminho_arquivo)
            else:
                print("Não há dados para salvar.")

        elif opcao == "6":
            # Reinicia os dados
            confirmar = input("Tem certeza que deseja reiniciar os dados? (s/n): ").lower()
            if confirmar == "s":
                matrizes = []
                print("Dados reiniciados.")

        elif opcao == "7":
            # Encerra o programa
            if matrizes:
                salvar_em_csv(matrizes, caminho_arquivo)
            print("Programa encerrado.")
            break

        elif opcao == "8":
            # Adicionar de 1 a 10 repetições a mais e depois prever a próxima repetição
            adicionar = solicitar_numero_repeticoes("Quantas repetições a mais você deseja adicionar (1 a 10)? ", 1, 10)
            numero_repeticoes += adicionar
            # Preenche as novas repetições adicionadas, se necessário
            if len(matrizes) < numero_repeticoes:
                preencher_repeticoes(matrizes, numero_repeticoes, numero_linhas)
            else:
                print("Todas as repetições já foram preenchidas.")
            # Após preencher, tenta prever a próxima (sem adicionar à lista)
            prever_repeticao(matrizes)

        elif opcao == "9":
            # Carregar dados de um arquivo CSV
            caminho_csv = input("Digite o caminho completo do arquivo CSV: ")
            matrizes_carregadas = carregar_de_csv(caminho_csv)
            if matrizes_carregadas:
                matrizes = matrizes_carregadas
                numero_repeticoes = len(matrizes)  # Atualiza o número de repetições para refletir o arquivo carregado
                print("Dados carregados e prontos para previsão ou edição.")

        else:
            print("Opção inválida. Tente novamente.")

# Executa o programa
if __name__ == "__main__":
    gerar_matriz()



Menu Principal:
1. Preencher dados das repetições
2. Editar dados de uma repetição já preenchida
3. Listar todas as repetições digitadas
4. Obter a previsão da próxima repetição
5. Salvar dados
6. Reiniciar os dados
7. Sair
8. Adicionar de 1 a 10 repetições a mais e depois prever a próxima repetição
9. Carregar dados de um arquivo CSV
